<a href="https://colab.research.google.com/github/arcila-heisman/ml_introductory_labs/blob/master/Lab_5/Laboratorio_5_parte_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Laboratorio 5 - Parte 2

### Máquinas de Vectores de Soporte

### 2019-I

#### Profesor: Julián D. Arias Londoño
#### julian.ariasl@udea.edu.co

Modificado por Heisman Arcila. (Versión original en: https://github.com/jdariasl/ML_IntroductoryCourse/tree/master/Labs)

## Guía del laboratorio

En esta archivo va a encontrar tanto celdas de código cómo celdas de texto con las instrucciones para desarrollar el laboratorio.

Lea atentamente las instrucciones entregadas en las celdas de texto correspondientes y proceda con la solución de las preguntas planteadas.

Nota: no olvide ir ejecutando las celdas de código de arriba hacia abajo para que no tenga errores de importación de librerías o por falta de definición de variables.

#### Primer Integrante: Danny Hernandez
#### C.C: 1105784633
#### Segundo Integrante:
#### C.C:

In [0]:
from __future__ import division
import numpy as np
import math
import matplotlib.pyplot as plt
#Algunas advertencias que queremos evitar
import warnings
warnings.filterwarnings("ignore")

## Ejercicio 1

En este ejercicio usaremos la regresión por vectores de soporte para resolver el problema de regresión de la base de datos AirQuality (https://archive.ics.uci.edu/ml/datasets/Air+Quality).

La siguiente celda de código limpia la base de datos de todos sus datos faltantes y la deja lista en la variable DataBase.

In [0]:
#cargamos la bd de entrenamiento
db = np.loadtxt('/content/AirQuality.data',delimiter='\t')  # Assuming tab-delimiter
print ("Dim de la base de datos original: " + str(np.shape(db)))
db = db.reshape(9357,13)
DataBase = db
#La base d edatos AirQuality tiene muchos valores perdidos, vamos a quitar todos esos registros de la base de datos
#Están marcados como -200. Donde haya un valor -200 eliminaremos el registro.

j = np.sum(db[:,12]==-200)
DataBase= db[db[:,12]!=-200]

print ("\nHay " + str(j) + " valores perdidos en la variable de salida. :(")

print ("\nDim de la base de datos sin las muestras con variable de salida perdido "+ str(np.shape(DataBase)))

#Ya hemos eliminado los registros con valor de la variable de salida perdido.

#Ahora vamos a imputar los valores perdidos en cada una de las características
print ("\nProcesando imputación de valores perdidos en las características . . .\n")
          
for i in range(0,13):
    col =  DataBase[:,i]
    media = round(np.mean(col))
    DataBase[:,i] = np.where(col==-200, media, col)

print ("Imputación finalizada.\n")

if any(DataBase[DataBase==-200]):
    print("Hay valores perdidos")
else:
    print ("No hay valores perdidos en la base de datos. Ahora se puede procesar. La base de datos está en la variable DataBase")

OSError: ignored

#Base de datos Julian

In [0]:
#cargamos la bd de entrenamiento
db = np.loadtxt('/content/AirQuality.data',delimiter='\t')  # Assuming tab-delimiter
print ("Dim de la base de datos original: " + str(np.shape(db)))
db = db.reshape(9357,13)
DataBase = db
#La base d edatos AirQuality tiene muchos valores perdidos, vamos a quitar todos esos registros de la base de datos
#Están marcados como -200. Donde haya un valor -200 eliminaremos el registro.
j = 0
for i in range(0,np.size(db,0)):
    if -200 == db[i,12]:
        #print i
        j+=1
        DataBase = np.delete(DataBase,i,0)
    
print ("\nHay " + str(j) + " valores perdidos en la variable de salida. :(")

print ("\nDim de la base de datos sin las muestras con variable de salida perdido "+ str(np.shape(DataBase)))

#Ya hemos eliminado los registros con valor de la variable de salida perdido.

#Ahora vamos a imputar los valores perdidos en cada una de las características
print ("\nProcesando imputación de valores perdidos en las características . . .\n")

#data = open("AirQuality_cleaned.data", "w")

for k in range(0,np.size(DataBase,0)):
    for w in range(0,13):
        if -200 == DataBase[k,w]:
            DataBase[k,w] = round(np.mean(DataBase[:,w]))
        
    #data.write(str(DataBase[k,:]))

#data.close()
print ("Imputación finalizada.\n")

hay_missed_values = False
for i in range(0,np.size(DataBase,0)):
    if -200 in DataBase[i,:]:
        hay_missed_values = True
if(hay_missed_values):
    print ("Hay valores perdidos")
else:
    print ("No hay valores perdidos en la base de datos. Ahora se puede procesar. La base de datos está en la variable DataBase")

Dim de la base de datos original: (9357, 13)

Hay 366 valores perdidos en la variable de salida. :(

Dim de la base de datos sin las muestras con variable de salida perdido (8991, 13)

Procesando imputación de valores perdidos en las características . . .

Imputación finalizada.

No hay valores perdidos en la base de datos. Ahora se puede procesar. La base de datos está en la variable DataBase


In [0]:
X = DataBase[:,0:12]

Y = DataBase[:,12]

Definimos la función

In [0]:
#Mean Absolute Percentage Error para los problemas de regresión
def MAPE(Y_est,Y):
    ind = Y != 0 #Remueve los elementos que son cero en la variable deseada
    N = np.size(Y[ind])
    mape = np.sum(abs((Y_est[ind].reshape(N,1) - Y[ind].reshape(N,1))/(Y[ind].reshape(N,1)+np.finfo(np.float).eps)))/N
#     mape = np.sum(abs((Y_est[ind].reshape(N,1) - Y[ind].reshape(N,1))/(Y[ind].reshape(N,1))))/N
    return mape

Complete el código siguiente para usar la regresión por vectores de suporte usando la librería sklearn. https://scikit-learn.org/stable/modules/generated/sklearn.svm.SVR.html

In [0]:
from sklearn.model_selection import KFold
from sklearn import preprocessing
from sklearn.svm import SVR
import time
#Complete el código para crear el modelo SVM para regresión. 
#Use un kernel rbf con una malla de valores así: C en {0.1, 100} y gamma en {0.0001, 0.1}

#clf = ...
#clf = SVR(kernel='rbf', gamma=0.1, C=100)
#clf.fit(X, Y) 

#Implemetamos la metodología de validación cross validation con 10 folds
Folds = 5
porcentaje_mean=np.zeros(Folds)
Errores = np.ones(Folds)
tiempo_i = time.time()
kf = KFold(n_splits=Folds)
#EficienciaVal = np.zeros(Folds)
def invento(kernelio, ceste, gammeste):
  j = 0
  for train_index, test_index in kf.split(X):

      X_train, X_test = X[train_index], X[test_index]
      y_train, y_test = Y[train_index], Y[test_index]  

      #Complete el código

      #Normalizamos los datos
      scaler = preprocessing.StandardScaler().fit(X_train)
      Xtrain = scaler.transform(X_train)
      Xtest = scaler.transform(X_test)

      # Entrenar el modelo
      #modelo = ...
      modelo = SVR(kernel=kernelio, gamma=gammeste, C=ceste, cache_size=200)
      modelo.fit(Xtrain, y_train)
      
      porcentaje_mean[j] = (len(modelo.support_vectors_)/len(Xtrain))
      # Validación del modelo
      #ypred = ...
      ypred = modelo.predict(Xtest)

      #porcentaje
      
      #EficienciaVal[j] = np.mean(ypred.ravel() == ypred.ravel())
      Errores[j] = MAPE(ypred, y_test)

      j+=1

  print ("\nTiempo total de ejecución: " + str(time.time()-tiempo_i) + " segundos.")
  print("\nError de validación: " + str(np.mean(Errores)) + " +/- " + str(np.std(Errores)))
  #print('Eficiencia durante la validación = ' + str(np.mean(EficienciaVal)) + '+-' + str(np.std(EficienciaVal)))
  print ("\n % Vectores de Soporte: " + str(np.mean(porcentaje_mean)))
  
  return np.mean(Errores), np.std(Errores), np.mean(porcentaje_mean), y_test, ypred

EX=invento('rbf',100, 1)


Tiempo total de ejecución: 7.8858864307403564 segundos.

Error de validación: 0.2259390395748797 +/- 0.11094914643434282

 % Vectores de Soporte: 0.21910801302774435


In [0]:
EX=invento('linear',100, 'auto')


Tiempo total de ejecución: 5595.462719678879 segundos.

Error de validación: 0.28853639336068776 +/- 0.1531772568363643

 % Vectores de Soporte: 0.3863047005129662


## Ejercicio 2

Una vez complete el código, realice las simulaciones necesarias para llenar la tabla siguiente:

In [0]:
import pandas as pd
#import qgrid
df_types = pd.DataFrame({
    'Kernel' : pd.Series(['linear','linear','linear','linear','linear','linear','rbf','rbf','rbf','rbf','rbf','rbf','rbf','rbf','rbf','rbf','rbf','rbf','rbf','rbf','rbf','rbf','rbf','rbf']),
    'C' : pd.Series([0.001,0.01,0.1,1,10,100,0.001,0.001,0.001,0.01,0.01,0.01,0.1,0.1,0.1,1,1,1,10,10,10,100,100,100]),
    'gamma' : pd.Series(['auto','auto','auto','auto','auto','auto',0.01,0.1,1,0.01,0.1,1,0.01,0.1,1,0.01,0.1,1,0.01,0.1,1,0.01,0.1,1])})
df_types["MAPE Promedio"] = ""
df_types["Intervalo de confianza"] = ""
df_types["% de Vectores de Soporte"] = ""
df_types.set_index(['Kernel','C','gamma'], inplace=True)
#Escriba acá su código
for i in df_types.index:
    print(i[0])
    ED=invento(i[0], i[1], i[2])
    df_types["MAPE Promedio"][i] = float(ED[0])
    df_types["Intervalo de confianza"][i] = float(ED[1])
    df_types["% de Vectores de Soporte"][i] = float(ED[2])

# Valores de referencia
# df_types["MAPE Promedio"][23] = "0.2259"
# df_types["Intervalo de confianza"][23] = "0.1109"
# df_types["% de Vectores de Soporte"][23] = "0.2191"

#qgrid_widget = qgrid.show_grid(df_types, show_toolbar=False)
#qgrid_widget

linear

Tiempo total de ejecución: 19.95706081390381 segundos.

Error de validación: 0.21532161504323985 +/- 0.06460100562932213

 % Vectores de Soporte: 0.3786300277723352
linear

Tiempo total de ejecución: 25.7901713848114 segundos.

Error de validación: 0.24490897173183038 +/- 0.0962859267563447

 % Vectores de Soporte: 0.37665612594249104
linear

Tiempo total de ejecución: 39.099332332611084 segundos.

Error de validación: 0.27680077418621385 +/- 0.13461740462666608

 % Vectores de Soporte: 0.384970000032475
linear

Tiempo total de ejecución: 98.13184905052185 segundos.

Error de validación: 0.2869582909645992 +/- 0.15071982828475336

 % Vectores de Soporte: 0.38622128608226974
linear


KeyboardInterrupt: ignored

Ejecute la siguiente instrucción para dejar guardados en el notebook los resultados de las pruebas.
#No funciona QGRID

In [0]:
df_types

In [0]:
qgrid_widget.get_changed_df()

MAPE Promedio Intervalo de confianza  \
Kernel C       gamma                                        
lineal 0.001   0.00                                         
       0.010   0.00                                         
       0.100   0.00                                         
       1.000   0.00                                         
       10.000  0.00                                         
       100.000 0.00                                         
rbf    0.001   0.01                                         
               0.10                                         
               1.00                                         
       0.010   0.01                                         
               0.10                                         
               1.00                                         
       0.100   0.01                                         
               0.10                                         
               1.00                                         
       1.000   0.01                                         
               0.10                                         
               1.00                                         
       10.000  0.01                                         
               0.10                                         
               1.00                                         
       100.000 0.01                                         
               0.10                                         
               1.00         0.2259                 0.1109   

                     % de Vectores de Soporte  
Kernel C       gamma                           
lineal 0.001   0.00                            
       0.010   0.00                            
       0.100   0.00                            
       1.000   0.00                            
       10.000  0.00                            
       100.000 0.00                            
rbf    0.001   0.01                            
               0.10                            
               1.00                            
       0.010   0.01                            
               0.10                            
               1.00                            
       0.100   0.01                            
               0.10                            
               1.00                            
       1.000   0.01                            
               0.10                            
               1.00                            
       10.000  0.01                            
               0.10                            
               1.00                            
       100.000 0.01                            
               0.10                            
               1.00                    0.2191

2.1 ¿Cuál es la finalidad de usar las funciones kernel en el modelo SVM?

R/: Se usan funciones kernel en el modelo SVM para poder ver el espacio entre caraceristicas con una dimension mayor para que la función lineal se acerque mas a los puntos.

2.2 ¿En este caso el porcentaje de vectores de soporte provee una información similar que en el problema de clasificación? Explique su respuesta.

R/: Segun el caso de la calidad del aire, este presenta datos linealmente separables con una cantidad menor de vectores, asi obtenemos mejor eficiencia, por ello se nora una relación entre eficiencia y el porcentaje y por tanto creemos que no se obtiene una información similar que el problema de clasificación.

2.3 Realice una gráfica de las salidas reales vs las predicciones del modelo SVM, para evaluar visualmente el desempeño del mismo. Esto solo para la configuración en la cuál se encontró el menor error.

In [0]:
'''Complete el código para hacer la gráfica aquí'''

In [0]:
este=invento('rbf', 100, 1)
plt.title('Salidas reales vs Predicciones', fontsize=14)
plt.plot(este[3])
plt.plot(este[4])
plt.legend(['Teórico', 'Predicho'])
plt.show()
print('Costo computacional: ', time.time()-tiempo_i, ' s')